<a href="https://colab.research.google.com/github/hillelda/ANLP/blob/main/rec_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title pip install
! pip install datasets
! pip install evaluate
! pip install accelerate -U
! pip install transformers[torch]
! pip install torch
! pip install peft
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
# @title Imports
import evaluate
import numpy as np
from datasets import load_dataset
import transformers
from transformers import (AutoModelForSequenceClassification, AutoTokenizer)
import torch
from tqdm import tqdm

In [3]:
# @title Globals
MODE = 'gemma' #@param ["regular", "lora", "large", "gemma"]

In [4]:
# @title load model
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
if MODE == 'regular':
  model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').cuda()
elif MODE == 'lora':
  model2 = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').cuda()
elif MODE == 'large':
  deberta = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-large').cuda()
elif MODE == 'gemma':
  !huggingface-cli login # ask for token for gemma
  gemma = AutoModelForSequenceClassification.from_pretrained('google/gemma-2b').cuda()
  gemma_tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [5]:
# @title data
def preprocess_function(examples):
    result = tokenizer(examples['sentence1'], examples['sentence2'], max_length=256, truncation=True, padding='max_length')
    return result

raw_datasets = load_dataset("nyu-mll/glue", 'mrpc')
raw_datasets = raw_datasets.map(preprocess_function,batched=True)

train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["validation"]

# train_dataset = train_dataset.select(range(300)) #training on 5k samples

# Set format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
# @title Metric
metric = evaluate.load("accuracy",)

# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return metric.compute(predictions=preds, references=p.label_ids)

def compute_metrics(preds, labels):
    pred_labels = np.argmax(preds, axis=1)
    return metric.compute(predictions=pred_labels, references=labels)

In [7]:
# @title Imports for Trainer alternative
from torch.utils.data import DataLoader
from torch.optim import Adam
from transformers import DataCollatorWithPadding
from transformers import get_scheduler


In [8]:
# from transformers import get_scheduler


# # @title Init trainer
# # training_args = TrainingArguments(output_dir='/tmp/', do_eval=True, do_train=True, num_train_epochs=3, per_device_train_batch_size=8, learning_rate =5e-5)
# # trainer = Trainer(
# #     model=model,
# #     args=training_args,
# #     train_dataset=train_dataset,
# #     eval_dataset=eval_dataset,
# #     compute_metrics=compute_metrics,
# #     tokenizer=tokenizer,
# # )


# def train(model, train_dataset, eval_dataset, tokenizer, num_epochs=5, learning_rate=5e-5, batch_size=16):
#     model.train()
#     model.cuda()
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     optim = Adam(model.parameters(), lr=learning_rate)
#     scheduler = get_scheduler(
#         "linear",
#         optim,
#         # num_warmup_steps=0,
#         num_training_steps=num_epochs * len(train_loader),
#         num_warmup_steps = int(0.1 * num_epochs * len(train_loader))
#     )
#     for epoch in range(num_epochs):
#         torch.cuda.empty_cache()
#         for batch in tqdm(train_loader):
#             optim.zero_grad()
#             input_ids = batch['input_ids'].cuda()
#             attention_mask = batch['attention_mask'].cuda()
#             labels = batch['label'].cuda()
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             loss.backward()
#             optim.step()
#             scheduler.step()
#         print("Epoch: " + str(epoch) + " - Loss: " + str(loss.item()))
#         model.eval()
#         eval_loader = DataLoader(eval_dataset, batch_size=batch_size)
#         all_preds = []
#         all_labels = []
#         for batch in tqdm(eval_loader):
#             input_ids = batch['input_ids'].cuda()
#             attention_mask = batch['attention_mask'].cuda()
#             labels = batch['label'].cuda()
#             with torch.no_grad():
#                 outputs = model(input_ids, attention_mask=attention_mask)
#             preds = torch.argmax(outputs.logits, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         accuracy = (all_preds == all_labels).mean()
#         print("Epochs: " + str(epoch + 1) + " - Learning Rate: " + str(learning_rate) + " - Batch Size: " + str(batch_size) + " - Accuracy: " + str(accuracy))
#     return model, {'accuracy': accuracy}

In [9]:
def train(model, train_dataset, eval_dataset, tokenizer, num_epochs=3, learning_rate=1e-5, batch_size=8):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            torch.cuda.empty_cache() # helps run on the collab GPU without OOM errors.
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            loss.backward() # Backpropogation
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        eval_accuracy = 0
        eval_loss = 0
        with torch.no_grad(): # only evaluating, don't change weights.
            for batch in eval_loader:
                torch.cuda.empty_cache()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                eval_loss += loss.item()
                pred = torch.argmax(outputs.logits, dim=1)
                eval_accuracy += (pred == labels).sum().item()

        eval_accuracy = eval_accuracy / len(eval_dataset)
        train_loss = train_loss / len(train_loader)
        eval_loss = eval_loss / len(eval_dataset)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}, Eval Loss: {eval_loss:.4f}, LR: {learning_rate}, Batch Size: {batch_size}")

    final_accuracy = eval_accuracy
    return model, {'accuracy': final_accuracy}

In [10]:
# @title Train!
# Train the model
if MODE == 'regular':
  trained_model, metrics = train(model, train_dataset, eval_dataset, tokenizer)
  metrics

In [11]:
torch.cuda.empty_cache() # clear the cache befoer next step

In [12]:
#@title Lora imports
import peft
from peft import LoraModel, LoraConfig

In [13]:
# print(model)

In [14]:
# @title Lora model config

lora_config_lora = LoraConfig(
    r=16,
    lora_alpha=32, #should be about r*2.
    lora_dropout=0.05,
    task_type="classification",
    target_modules=['query_proj', 'value_proj'], # learn on target modules with LoRA
    # do learn the sequence classification head and the pooler weights
    modules_to_save=['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
)


In [15]:
# print(lora_model)

In [16]:
# @title Train LORA
if MODE == 'lora':
  lora_model = LoraModel(model2, lora_config_lora, adapter_name="default")
  trained_model, metrics = train(lora_model, train_dataset, eval_dataset, tokenizer, learning_rate=1e-4)
  metrics

In [17]:
# @title define deberta
if MODE == 'large':
  lora_config_deberta = LoraConfig(
    r=16,
    lora_alpha=32, #should be about r*2.
    lora_dropout=0.05,
    task_type="classification",
    target_modules=['query_proj', 'value_proj'],
    modules_to_save=['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
)
  lora_model_large = LoraModel(deberta, lora_config_deberta, adapter_name="default")

In [18]:
# @title Train deberta large
if MODE == 'large':
  trained_model, metrics = train(lora_model_large, train_dataset, eval_dataset, tokenizer, learning_rate=1e-4)
  metrics

In [19]:
print(gemma) # neaded so we can find the layer names in new model.

GemmaForSequenceClassification(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    

In [20]:
# @title Deifne gemma
if MODE == 'gemma':
  lora_config_gemma = LoraConfig(
    r=16,
    lora_alpha=32, #should be about r*2.
    lora_dropout=0.05,
    task_type="classification",
    target_modules=['q_proj', 'v_proj'], # other layer names in this model
    modules_to_save=['pooler', 'classifier']
)
  lora_model_gemma = LoraModel(gemma, lora_config_gemma, adapter_name="default")

In [ ]:
# @title Train gemma
if MODE == 'gemma':
  trained_model, metrics = train(lora_model_gemma, train_dataset, eval_dataset, gemma_tokenizer, learning_rate=5e-5, batch_size=4)
  metrics

Epoch 1/3: 100%|██████████| 917/917 [35:10<00:00,  2.30s/it]


Epoch 1/3 - Train Loss: 0.6211, Eval Accuracy: 0.6814, Eval Loss: 0.1490, LR: 5e-05, Batch Size: 4


Epoch 2/3:  11%|█         | 98/917 [03:46<31:37,  2.32s/it]

In [ ]:
# @title Evaluate
# metrics = trainer.evaluate(eval_dataset=eval_dataset)
# metrics